<CENTER>
</br>
<p><font size="5"> Nathan Sanglier </span></p>
<p><font size="4">  Unconstrained Optimization </font></p>
<p></p>
<p><font size="5"> Descent Algorithms for Unconstrained Optimization </font></p>
</p></br>
</p>
</CENTER>

----------------------------

## <center>  <span style="color:#FF0000"> NOT FINISHED YET </span> </center>

##  <span style="color:#00B8DE"> 0 - Introduction </span>

> The goal of this notebook is to test 1st-order (requires the gradient of objective function) and 2nd-order (requires the gradient and the Hessian matrix of objective function) descent algorithms. We will apply these algorithms to quadaratic and non-quadratic functions (when possible), in Julia programming language. If you want to see the plots and LaTeX equations appear well, please download and run the notebook.

#### <span style="color:#DAF7A6">Imports </span>

In [6]:
using Plots
using LinearAlgebra: norm, cond, diagm

##  <span style="color:#00B8DE"> I - Gradient Descent </span>

> $\underline{Local\ Descent}$ : To find the local minimum of a function $f : \mathbb{R}^n \rightarrow \mathbb{R}$, we design a sequence $(x_k)_{k \geq 0}$ with $x_{k+1} = x_k + {\rho}_k d_k$. $\\$
We have from Taylor expansion at first order: $f(x_{k+1}) = f(x_k) + \rho_k {d_k}^T \nabla f(x_k) + o(\rho_k)$. Moreover, we need $f(x_{k+1}) \leq f(x_k)$, i.e. $\rho_k {d_k}^T \nabla f(x_k) \leq 0$. $\\$

> $\underline{Gradient\ Descent}$ : Local Descent with $\rho_k \geq 0$ and $d_k = - \nabla f(x_k)$ because $- \rho_k {\lVert \nabla f(x_k) \rVert}^2 \leq 0$.

#### <span style="color:#DAF7A6">I. 1. Gradient Descent with Constant Stepsize </span>

> A first naive way to find a stepsize is to choose and set a constant stepsize.

> Let's implement the Gradient Descent with constant stepsize.

In [2]:
function gradient_descent_constant_stepsize(init_pt, ρ, ∇f, opt_f, max_iter, precision)

    pt = init_pt
    nb_iter = 0
    res = pt
    
    while norm(pt - opt_f) > precision && nb_iter < max_iter
        
        d = - ∇f(pt)
        pt += ρ * d
        res  = [res pt]
        nb_iter += 1
    end

    return res, nb_iter
end;

#### <span style="color:#DAF7A6">I. 2. Gradient Descent with Backtracking </span>

> A way to find a good stepsize $\rho$ is to use a backtracking line search (inexact line search).

> $\underline{Backtracking\ Line\ Search}$ :
Let us have 2 constants $\alpha$, $\beta$ with $0 < \alpha < 0.5$ and $0 < \beta < 1$, and let us first take $\rho_k = 1$. $\\$
While $f(x_k + \rho_k d_k) > f(x_k) + \alpha \rho_k \nabla f(x_k)^T d_k$, we will update $\rho_k$ by $\rho_k = \beta \rho_k$. The term $f(x_k) + \alpha \rho_k \nabla f(x_k)^T d_k$ is the linear extrapolation of $f$ with a slope of $\alpha$ smaller.
>
> In fact, since $d_k$ is a descent direction, $\nabla f(x_k)^T d_k \leq 0$, when $\rho_k$ is small enough, we will get $f(x_k + \rho_k d_k) = f(x_k) + \rho_k \nabla f(x_k)^T d_k < f(x_k) + \alpha \rho_k \nabla f(x_k)^T d_k$ (this is known as the Armijo condition). Thus, with the backtracking line search, we restrict our domain search for $\rho_k$ and it is called backtracking because we start with a unit stepsize and and then reduce it by a factor $\beta$.

> Let's implement the Gradient Descent with Backtracking.

In [3]:
function backtracking(init_pt, f, ∇f, d, α, β)
    
    t = 1
    nb_iter = 0

    pt = init_pt + t * d
    while f(pt) > f(init_pt) + α * t * ∇f(init_pt)' * d

        t *= β
        pt = init_pt + t * d
        nb_iter += 1
    end
    
    return pt, nb_iter
end;

In [4]:
function gradient_descent_backtracking(init_pt, f, ∇f, α, β, opt_f, max_iter, precision)

    pt = init_pt
    nb_iter = 0
    nb_inner_iter = 0
    res = pt
    
    while norm(pt - opt_f) > precision && nb_iter < max_iter
        
        d = - ∇f(pt)
        pt, nb_iter_backtrack = backtracking(pt, f, ∇f, d, α, β)
        nb_inner_iter += nb_iter_backtrack
        res  = [res pt]
        nb_iter += 1
    end

    return res, nb_iter, nb_inner_iter
end;

#### <span style="color:#DAF7A6">I. 3. Gradient Descent with Optimal Stepsize for Quadratic Functions </span>

> We can find the Gradient Descent optimal stepsize easily $\textbf{only for a quadratic function}$.

> $\underline{Quadratic\ Function}$ : Let us have $A \in \R^{n \times n}$, $b \in \R^n$ and $c \in \R$. $\\$
$\begin{array}{l|rcl} 
f : & \R^n & \longrightarrow & \R \\
    & x & \longmapsto & x^T A x + x^T b + c \end{array}$ $\\$ is a quadratic function.

> $\underline{Optimal\ Stepsize\ Quadratic\ Function}$ : $\\$
Let us have the quadratic function $f(x) = \frac{1}{2} x^T A x + x^T b + c$ with $A \succ 0$ (positive definite, i.e. all the eigenvalues of $A$ are $> 0$, i.e. $\forall x$, $x^T A x > 0$) and symmetric (i.e. $A^T$ = $A$). It also works with $A \prec 0$, but it will be a max instead of a min, and be careful with the signs. $\\$
The optimal stepsize is $\rho_k = \frac{{\lVert A x_k + b \rVert}_2^2}{{\lVert A x_k + b \rVert}_A^2}$.

> $\underline{Proof}$ : We want to minimize $f(x_{k+1}) = f(x_k - \rho_k \nabla f(x_k))$, with $\nabla f(x_k) = A x + b$. $\\$
$f(x_{k+1}) = \frac{1}{2} (x_k - \rho_k \nabla f(x_k))^T A (x_k - \rho_k \nabla f(x_k)) + (x_k - \rho_k \nabla f(x_k))^T b + c$. $\\$
Let us derive this expression w.r.t. $\rho_k$ : $\frac{d f(x_{k+1})}{d \rho_k} = \rho_k \nabla f(x_k)^T A \nabla f(x_k) - \nabla f(x_k)^T \nabla f(x_k)$. $\\$
So, $\frac{d f(x_{k+1})}{d \rho_k} = 0 \Leftrightarrow \rho_k = \frac{{\lVert \nabla f(x_k) \rVert}_2^2}{{\lVert \nabla f(x_k) \rVert}_A^2} = \frac{{\lVert A x_k + b \rVert}_2^2}{{\lVert A x_k + b \rVert}_A^2}$.

> Let's implement the Gradient Descent with optimal stepsize for quadratic functions.

In [5]:
function gradient_descent_optimal_stepsize_quad(init_pt, ∇p, A, b, opt_p, max_iter, precision)

    pt = init_pt
    nb_iter = 0
    res = pt
    
    while norm(pt - opt_p) > precision && nb_iter < max_iter

        d = - ∇p(pt, A, b)
        t = norm(∇p(pt, A, b))^2 / (∇p(pt, A, b)^T * A * ∇p(pt, A, b))
        pt += t * d
        res  = [res pt]
        nb_iter += 1
    end

    return res, nb_iter
end;

##  <span style="color:#00B8DE"> II - Conjugate Gradient Descent </span>

#### <span style="color:#DAF7A6">II. 1. Conjugate Gradient Descent for Quadratic Functions </span>

> $\underline{Property}$ : For the Gradient Descent with optimal stepsize for quadratic functions, one can check that successive descent directions are orthogonal. $\\$
In fact, we have $\frac{d f(x_{k+1})}{d \rho_k} = 0 \Leftrightarrow d_k^T \nabla f(x_{k+1}) = d_k^T d_{k+1} = 0$.
>
> This is clearly a problem as successive approximations of the solution form a zig-zagging trajectory. The Conjugate Gradient Descent helps combatting this effect, by having a set of directions that are conjugate w.r.t. $A$.

> $\underline{Conjugate\ Vectors\ w.r.t.\ A}$ : set of non-zero directions $(d_k)_{k=0:n-1}$ such that $\forall i \neq j$, $d_i^T A d_j = 0$.

> $\underline{Property}$ : Let us have the quadratic function $f(x) = \frac{1}{2} x^T A x - x^T b$ with $A \succ 0$, and $(d_k)_{k=0:n-1}$ be a set of conjugate vectors w.r.t. $A$. Then, $\\$
> - $(d_k)_{k=0:n-1}$ is a basis of $\R^n$
>
> - $x^* = \sum_{k=0}^{n-1} \frac{d_k^T b}{{\lVert d_k \rVert}_A^2} d_k$ yields the min of $f$.
>
> - for $x_0 \in \R^n$, the sequence $x_{k+1} = x_k - \frac{d_k^T \nabla f(x_k)}{{\lVert d_k \rVert}_A^2} d_k$ yields $x_n = x^*$.

> $\underline{Proof}$ : $\\$
> - Let $(\lambda_k)_{k=0:n-1}$ such that $\sum_{k=0}^{n-1} c_k d_k = 0$, and let us prove that $\forall k \in 0 : n-1$, $c_k = 0$. $\\$
In fact, an implication of the Steinitz Exchange Lemma is that a linearly independent set $L$ is a basis $\Leftrightarrow$ it is maximal, i.e. it is not a proper subset of any linearly independent set. So here, we just have to show that the $c_k$'s are linearly independent since the cardinal of this set is n, i.e. the dimension of $\R^n$. $\\$
$\forall i \in 0:n-1$, $d_i^T A (\sum_{k=0}^{n-1} c_k d_k) = c_i {\lVert d_k \rVert}_A^2 = 0 \Leftrightarrow c_i = 0$ since $d_i \neq 0$ and $A \succ 0$. So, $(d_k)_{k=0:n-1}$ is a basis of $\R^n$.
>
> - $\forall i \in 0:n-1$, we have 
\begin{align*}
d_i^T \nabla f(x^*) &= d_i^T (A [\sum_{k=0}^{n-1} \frac{d_k^T b}{{\lVert d_k \rVert}_A^2} d_k] - b) \\
&= \frac{d_i^T b}{{\lVert d_k \rVert}_A^2} \cdot d_i^T A d_i - d_i^T b \\
&= d_i^T b - d_i^T b \\
&= 0 \\
\end{align*}
Thus, as $d_i \neq 0$, we have $\nabla f(x^*) = 0$, so $x^*$ yields the min of $f$.
>
> - We have $x_n = x_0 + \sum_{k=0}{n-1} (x_{k+1} - x_k)$ and $x_{k+1} - x_k = - \frac{d_k^T \nabla f(x_k)}{{\lVert d_k \rVert}_A^2} d_k \propto d_k$. $\\$
\begin{align*}
\forall i \in 0:n-1 d_i^T \nabla f(x_n) &= d_i^T A x_n - d_i^T b \\
&= d_i^T A (x_{i+1} - x_i) + d_i^T A x_0 - d_i^T b \\
&= - \frac{d_i^T (A x_i - b)}{{\lVert d_k \rVert}_A^2} {\lVert d_k \rVert}_A^2 + d_i^T A x_0 - d_i^T b \\
&= - d_i^T (A (x_i - x_0) - b) - d_i^T b \\
&= - d_i^T (A \sum_{k=0}^{i-1}(x_{k+1} - x_k) - b) - d_i^T b \\
&= 0 + d_i^T b - d_i^T b \\
&= 0
\end{align*}
Thus, as $d_i \neq 0$, we have $\nabla f(x_n) = 0$, so $x_n = x^*$.

> Now, we must design such a set of conjugate directions. Starting from $d_0 = \nabla f(x_0)$, $\forall k \geq 1$, we will show that $d_k$ can be computed recursively from a linear combination of $\nabla f(x_k)$ and $d_{k-1}$ via the following algorithm.

> The first function we will try to find the minimum is $\\$
$\begin{array}{l|rcl} 
f : & \R \times \R & \longrightarrow & \R \\
    & (x, y) & \longmapsto & ((x - 2) \sin(y - 1))^2 + x^2 + y^2 \end{array}$ $\\$
>
> This function is not convex.

In [ ]:
f(x, y) = ((x - 2) * sin(y - 1))^2 + x^2 + y^2

# u = [x, y]
f(u) = ((u[1] - 2) * sin(u[2] - 1))^2 + u[1]^2 + u[2]^2

opt_f = [0.09172811859570773, 0.7789586896619789]; # optimal point of the function

println("Min f(x, y) = $(f(opt_f))")
println("Optimum at (x, y)* = ($(opt_f[1]), $(opt_f[2]))")

> Let's plot it.

In [ ]:
dx_f, dy_f = range(-3, 3, 100), range(-3, 3, 100)
dz_f = f.(dx_f', dy_f)

contourf(dx_f, dy_f, dz_f, size=(400, 350))
scatter!([opt_f[1]], [opt_f[2]], label="opt")
xlabel!("x")
ylabel!("y")
title!(raw"$f(x, y) = ((x - 2) \sin(y - 1))^2 + x^2 + y^2\$")

### <span style="color:#00B8DE"> I - Function plot </span>


In [ ]:
# u = [x, y]
∇f(u)  = [
    2 * sin(u[2] - 1)^2 * (u[1] - 2) + 2 * u[1],
    2 * (u[1] - 2)^2 * cos(u[2] - 1) * sin(u[2] - 1) + 2 * u[2]
];

In [ ]:

# Let's check if the gradient of this point is null
∇f(opt_f)

### <span style="color:#00B8DE"> II - Backtracking </span>

In [ ]:
init_pt = [-4, -2]
max_iter  = 10e4
precision = 1.e-3

res_grad_const, nb_iter_grad_const = gradient_descent_constant_stepsize(init_pt, 0.1, ∇f, opt_f, max_iter, precision)
res_grad_const_2, nb_iter_grad_const_2 = gradient_descent_constant_stepsize(init_pt, 0.01, ∇f, opt_f, max_iter, precision)

println("Nb of iterations for t = 0.1 : $nb_iter_grad_const")
println("Nb of iterations for t = 0.01 : $nb_iter_grad_const_2")

dx_f, dy_f = range(-6, 6, 100), range(-6, 6, 100)
dz_f = f.(dx_f', dy_f)
contourf(dx_f, dy_f, dz_f)
plot!(res_grad_const[1, :], res_grad_const[2, :], marker=:circle, label="Path for t=0.1", title="Gradient Descent with constant stepsize for f")
plot!(res_grad_const_2[1, :], res_grad_const_2[2, :], marker=:circle, label="Path for t=0.01", title="Gradient Descent with constant stepsize for f")
scatter!([opt_f[1]], [opt_f[2]], label="Minimum")

<span style="color:#DAF7A6">
We can see the impact of the stepsize !
If the stepsize is too small, then the convergence will take a very high number of iterations as we can see with t = 0.1
For t = 0.1, the step size seems good as the convergence happens quickly.
However, a constant stepsize is not an optimal choice: it would be intuitively better to begin with high stepsize, and then reduce it when we're near the optimum.

In [ ]:
precision = 1.e-3
max_iter  = 10e4
init_pt = [-4, -2]
a = 0.25
β = 0.7

res_grad_backtrack, nb_iter_grad_backtrack, nb_inner_iter_grad_backtrack = gradient_descent_backtracking(init_pt, f, ∇f, a, β, opt_f, max_iter, precision)

contourf(dx_f, dy_f, dz_f)
plot!(res_grad_backtrack[1, :], res_grad_backtrack[2, :], marker=:circle, label="Path", title="Gradient Descent with backtracking for f")
scatter!([opt_f[1]], [opt_f[2]], label="Minimum")

In [ ]:
println("Error with constant stepsize (t = 0.1): $(norm(res_grad_const[:, end] - opt_f))")
println("Number of iterations with constant stepsize (t = 0.1): $nb_iter_grad_const\n")
    
println("Error with Gradient + Backtracking: $(norm(res_grad_backtrack[:, end] - opt_f))")
println("Number of iterations for Gradient + Backtracking= $nb_iter_grad_backtrack")
println("Number of inner loops for Gradient + Backtracking = $nb_inner_iter_grad_backtrack")

<span style="color:#DAF7A6">
We can see that we have a lower number of global iterations for the Gradient + Backtrack method. However, the backtracking search for the stepsize is costly, and overall, this method is not as fast as the gradient with constant stepsize. However, with this method, we don't need to look manually for a good stepsize.

### <span style="color:#00B8DE"> III - Gradient and Newton </span>

In [ ]:
function ∇2f(x,y)
    d2f_dx2 = 2 * (sin(y - 1)^2 + 1)
    d2f_dxdy = 4 * cos(y - 1) * sin(y - 1) * (x - 2)
    d2f_dydx = 4 * (x - 2) * cos(y - 1) * sin(y - 1)
    d2f_dy2 = 2 * (x - 2)^2 * (cos(y - 1)^2 - sin(y - 1)^2) + 2
    hessian_mat = [
        [d2f_dx2 d2f_dxdy]
        [d2f_dydx d2f_dy2]
        ]
    return hessian_mat
end;

In [ ]:
# u = [x, y]
function ∇2f(u)
    d2f_dx2 = 2 * (sin(u[2] - 1)^2 + 1)
    d2f_dxdy = 4 * cos(u[2] - 1) * sin(u[2] - 1) * (u[1] - 2)
    d2f_dydx = 4 * (u[1] - 2) * cos(u[2] - 1) * sin(u[2] - 1)
    d2f_dy2 = 2 * (u[1] - 2)^2 * (cos(u[2] - 1)^2 - sin(u[2] - 1)^2) + 2
    hessian_mat = [
        [d2f_dx2 d2f_dxdy]
        [d2f_dydx d2f_dy2]
        ]
    return hessian_mat
end;

In [ ]:
# Newton Algorithm with Backtracking
#-------------------

function newton_backtracking(init_pt, f, ∇f, ∇2f, a, β, opt_f, max_iter, precision)

    """
    IN:
    ---
    init_pt: starting point
    f, ∇f, ∇2f: function to be minimized, its gradient and its Hessian matrix
    a, β: constants for backtracking
    opt_f: minimal point of the function
    max_iter: max nb of iterations
    precision: minimum precision to reach
    OUT:
    ----
    res: path of the algorithm
    nb_iter: number of global iterations
    nb_inner_iter: number of inner iterations due to backtracking
    """

    pt = init_pt
    nb_iter = 0
    nb_inner_iter = 0
    res = pt
    
    while norm(pt - opt_f) > precision && nb_iter < max_iter
        
        d = - inv(∇2f(pt)) * ∇f(pt)
        pt, nb_iter_backtrack = backtracking(pt, f, ∇f, d, a, β)
        nb_inner_iter += nb_iter_backtrack
        res  = [res pt]
        nb_iter += 1
    end

    return res, nb_iter, nb_inner_iter
end;

In [ ]:
precision = 1.e-3
max_iter  = 10e4
init_pt = [-4, -2]
a = 0.25
β = 0.7

res_newton_backtrack, nb_iter_newton_backtrack, nb_inner_iter_newton_backtrack = newton_backtracking(init_pt, f, ∇f, ∇2f, a, β, opt_f, max_iter, precision)

contourf(dx_f, dy_f, dz_f)
plot!(res_newton_backtrack[1, :], res_newton_backtrack[2, :], marker=:circle, label="Path", title="Newton Method with backtracking for f")
scatter!([opt_f[1]], [opt_f[2]], label="Minimum")

In [ ]:
println("Error with Newton + backtracking: $(norm(res_newton_backtrack[:, end] - opt_f))")
println("Number of iterations for Newton + backtracking= $nb_iter_newton_backtrack")
println("Number of inner loops for Newton + backtracking = $nb_inner_iter_newton_backtrack")

<span style="color:#DAF7A6">
The Newton algorithm is much faster than the gradient + backtracking method in terms of global iterations, but also in terms of iterations for backtracking. However, in more complex cases, the calculation of the inverse of the Hessian matrix could be difficult and costly.

### <span style="color:#00B8DE"> IV - Conjugate gradient </span>

In [ ]:
A = [
    [1 0]
    [0 5]
    ]

b=[1, 1]

p(x, y)  = 0.5 * [x, y]' * A * [x, y] - b' * [x, y]

# u = [x, y]
p(u, A, b)  = 0.5 * u' * A * u - b' * u

# u = [x, y]
∇p(u, A, b) = A * u - b

opt_p = inv(A) * b

dx_p, dy_p = range(-3, 3, 100), range(-3, 3, 100)
dz_p = p.(dx_p', dy_p)

contourf(dx_p, dy_p, dz_p, size=(400, 350))
scatter!([opt_p[1]], [opt_p[2]], label="opt")

In [ ]:
precision = 1.e-3
max_iter  = 10e4
init_pt = [-2, 3]

res_grad_opti, nb_iter_grad_opti = gradient_descent_optimal_stepsize(init_pt, ∇p, A, b, opt_p, max_iter, precision)

dx_p, dy_p = range(-4, 4, 100), range(-4, 4, 100)
dz_p = p.(dx_p', dy_p)
contourf(dx_p, dy_p, dz_p)
plot!(res_grad_opti[1, :], res_grad_opti[2, :], marker=:circle, label="Path", title="Gradient Descent with optimal stepsize for p")
scatter!([opt_p[1]], [opt_p[2]], label="Minimum")

In [ ]:
println("Error with gradient with optimal stepsize: $(norm(res_grad_opti[:, end] - opt_p))")
println("Number of iterations = $nb_iter_grad_opti")

<span style="color:#DAF7A6">
The method for gradient with optimal stepsize is a method that works well as we always converge to the minimum, but an exact optimal stepsize is easy to compute only for quadratic functions. Moreover, we have a problem of zizagging as the the direction we take at each step is orthogonal to the previous one.

In [ ]:
# Conjugate Gradient Algorithm (for Quadratic function !)
#-------------------

function conjugate_gradient_descent(init_pt, ∇p, A, b, opt_p, max_iter, precision)

    """
    IN:
    ---
    init_pt: starting point
    ∇p: gradient of the function to minimize
    A, b: parameters of the quadratic function
    opt_p: minimal point of the function
    max_iter: max nb of iterations
    precision: minimum precision to reach
    OUT:
    ----
    res: path of the algorithm
    nb_iter: number of iterations
    err: error values along the path
    """

    pt = init_pt
    nb_iter = 0
    res = pt
    err = norm(pt - opt_p)

    g = ∇p(pt, A, b)
    d = - ∇p(pt, A, b)
    alpha = - (g' * d) / (d' * A * d)
    
    while norm(pt - opt_p) > precision && nb_iter < max_iter

        pt += alpha * d
        g = ∇p(pt, A, b)
        beta = (g' * A * d) / (d' * A * d)
        d = - g + beta * d
        alpha = - (g' * d) / (d' * A * d)
        res  = [res pt]
        nb_iter += 1
        err = [err norm(pt - opt_p)]
    end

    return res, nb_iter, err
end;

In [ ]:
precision = 1.e-3
max_iter  = 10e4
init_pt = [-2, 3]

res_conj_grad, nb_iter_conj_grad, err_conj_grad = conjugate_gradient_descent(init_pt, ∇p, A, b, opt_p, max_iter, precision)

contourf(dx_p, dy_p, dz_p)
plot!(res_conj_grad[1, :], res_conj_grad[2, :], marker=:circle, label="Path", title="Conjugate Gradient Descent for p")
scatter!([opt_p[1]], [opt_p[2]], label="Minimum")

In [ ]:
println("Error with CG: $(err_conj_grad[end])")
println("Error number of iterations with CG: $(size(res_conj_grad)[2])")

<span style="color:#DAF7A6">
The conjugate gradient method is even much faster than the gradient with optimal stepsize, as we needed 8 iterations with the optimal stepsize. In fact, the zigzagging problem of the gradient with optimal stepsize has been solved and the minimum is always reached with this method. It is also important to notice that this implementation of the conjugate gradient algorithm is only valid for quadratic functions.

### A larger problem

In [ ]:
n1 = 10
A1 = randn(n1,n1)
A1_1 = A1' * A1
A1_2 = A1_1 + diagm(0=>ones(n1))
b1 = randn(n1)
opt1_1 = inv(A1_1)*b1
opt1_2 = inv(A1_2)*b1

n2 = 100
A2 = randn(n2,n2)
A2_1 = A2' * A2
A2_2 = A2_1 + diagm(0=>ones(n2))
b2 = randn(n2)
opt2_1 = inv(A2_1)*b2
opt2_2 = inv(A2_2)*b2;

In [ ]:
precision = 1.e-4
max_iter  = 10e4

res_conj_grad_1_1, nb_iter_conj_grad_1_1, err_conj_grad_1_1 = conjugate_gradient_descent(randn(n1), ∇p, A1_1, b1, opt1_1, max_iter, precision)
res_conj_grad_1_2, nb_iter_conj_grad_1_2, err_conj_grad_1_2 = conjugate_gradient_descent(randn(n1), ∇p, A1_2, b1, opt1_2, max_iter, precision)

println("cond(A1_1) = $(cond(A1_1))")
println("cond(A1_2) = $(cond(A1_2))")

plot(err_conj_grad_1_1[1, :]/norm(opt1_1),yscale=:log10,label="error norm with A1_1",size=(400,300))
plot!(err_conj_grad_1_2[1, :]/norm(opt1_2),yscale=:log10,label="error norm with A1_2",size=(400,300))

In [ ]:
res_conj_grad_2_1, nb_iter_conj_grad_2_1, err_conj_grad_2_1 = conjugate_gradient_descent(randn(n2), ∇p, A2_1, b2, opt2_1, max_iter, precision)
res_conj_grad_2_2, nb_iter_conj_grad_2_2, err_conj_grad_2_2 = conjugate_gradient_descent(randn(n2), ∇p, A2_2, b2, opt2_2, max_iter, precision)

println("cond(A1_1) = $(cond(A2_1))")
println("cond(A1_2) = $(cond(A2_2))")

plot(err_conj_grad_2_1[1, :]/norm(opt2_1),yscale=:log10,label="error norm with A2_1",size=(400,300))
plot!(err_conj_grad_2_2[1, :]/norm(opt2_2),yscale=:log10,label="error norm with A2_2",size=(400,300))

<span style="color:#DAF7A6">
We can see that for small n (n=10), the condition number of A has almost no impact on the performance of the Conjugate Gradient method. In fact, the evolution of the error with iterations is almost identical (see 1st graph). However, when it comes to larger problems, with n higher (n=100), then then condition number of A is very important! In fact, we can see on the second graph that if the condition number is too high, then the algorithm will converge very very slowly, whereas if we reduce the condition number of A, the error decreases much quicker.

### <span style="color:#00B8DE"> V - Polak-Ribière </span>

#### <span style="color:#00B8DE">Polack-Ribière method</span>

Initialization: 

> 1. initialize ${\bf x_0}$ and calculate $\Delta {\bf x_0}$, where $\Delta {\bf x_n}= - \nabla f ({\bf x_n})$ 
> 2. $\alpha_0 = \arg \min_\alpha f({\bf x_0}+\alpha \Delta {\bf x_0})$
> 3. ${\bf x_1}={\bf x_0}+\alpha_0 \Delta {\bf x_0}$

Iterations: 

> 1. $\beta_{n} = \frac{\nabla f ({\bf x_n})^{\top} (\nabla f ( {\bf x_n})-\nabla f ({\bf x_{n-1}}))} {\nabla f ({\bf x_{n-1})^{\top}} \nabla f ({\bf x_{n-1}})}$
> 2. ${\bf d_n}= -\nabla f ({\bf x_n})+\beta_n {\bf d_{n-1}}$
> 3. $\alpha_n=\arg \min_{\alpha} f({\bf x_n}+\alpha {\bf d_n})$
> 4. ${\bf x_{n+1}}={\bf x_n}+\alpha_{n} {\bf d_n}$


In [ ]:
# Polak-Ribière Method
#-------------------

function polak_ribiere(init_pt, f, ∇f, a, β, opt_f, max_iter, precision)

    """
    IN:
    ---
    init_pt: starting point
    f, ∇f: function to be minimized and its gradient
    a, β: constants for backtracking
    opt_f: minimal point of the function
    max_iter: max nb of iterations
    precision: minimum precision to reach
    OUT:
    ----
    res: path of the algorithm
    nb_iter: number of iterations
    """

    pt = init_pt
    nb_iter = 0
    nb_inner_iter = 0
    res = pt

    g = ∇f(pt)
    d = - ∇f(pt)
    
    while norm(pt - opt_f) > precision && nb_iter < max_iter

        pt, nb_iter_backtrack = backtracking(pt, f, ∇f, d, a, β)
        nb_inner_iter += nb_iter_backtrack
        g_new = ∇f(pt)
        beta = (g_new - g)' * g_new / (g' * g)
        d = - g_new + beta * d
        g = g_new
        res  = [res pt]
        nb_iter += 1
    end

    return res, nb_iter, nb_inner_iter
end;

In [ ]:
precision = 1.e-3
max_iter  = 10e4
init_pt = [-4, -2]
a = 0.25
β = 0.7

res_polak_ribiere, nb_iter_polak_ribiere, nb_inner_iter_polak_ribiere = polak_ribiere(init_pt, f, ∇f, a, β, opt_f, max_iter, precision)

res_grad_backtrack, nb_iter_grad_backtrack, nb_inner_iter_grad_backtrack = gradient_descent_backtracking(init_pt, f, ∇f, a, β, opt_f, max_iter, precision)

contourf(dx_f, dy_f, dz_f)
plot!(res_polak_ribiere[1, :], res_polak_ribiere[2, :], marker=:circle, label="Path PR with Backtracking")
plot!(res_grad_backtrack[1, :], res_grad_backtrack[2, :], marker=:circle, label="Path Gradient with Backtracking")
scatter!([opt_f[1]], [opt_f[2]], label="Minimum")

In [ ]:
println("Error with gradient = $(norm(res_grad_backtrack[:,end] - opt_f))")
println("Number of iterations = $nb_iter_grad_backtrack")
println("Number of inner loops = $nb_inner_iter_grad_backtrack\n")

println("Error with Polack-Ribière = $(norm(res_polak_ribiere[:,end] - opt_f))")
println("Number of iterations = $nb_iter_polak_ribiere")
println("Number of inner loops = $nb_inner_iter_polak_ribiere")

<span style="color:#DAF7A6">
We can see that for this initial point, the method of Polack-Ribière (it is an extension of the conjugate gradient method to non quadratic functions) is better than the Gradient + Backtracking method. In fact, it has a lower number of global iterations and of inner iterations due to backtracking ! It is logic since this method is more sophisticated.

### <span style="color:#00B8DE"> VI - Changing Initialization </span>

In [ ]:
precision = 1.e-3
max_iter  = 10e4
init_pt = [-5, -0.5]
a = 0.25
β = 0.7

res_polak_ribiere, nb_iter_polak_ribiere, nb_inner_iter_polak_ribiere = polak_ribiere(init_pt, f, ∇f, a, β, opt_f, max_iter, precision)

res_grad_backtrack, nb_iter_grad_backtrack, nb_inner_iter_grad_backtrack = gradient_descent_backtracking(init_pt, f, ∇f, a, β, opt_f, max_iter, precision)

contourf(dx_f, dy_f, dz_f)
plot!(res_polak_ribiere[1, :], res_polak_ribiere[2, :], marker=:circle, label="Path PR with Backtracking")
plot!(res_grad_backtrack[1, :], res_grad_backtrack[2, :], marker=:circle, label="Path Gradient with Backtracking")
scatter!([opt_f[1]], [opt_f[2]], label="Minimum")

In [ ]:
println("Error with gradient = $(norm(res_grad_backtrack[:,end] - opt_f))")
println("Number of iterations = $nb_iter_grad_backtrack")
println("Number of inner loops = $nb_inner_iter_grad_backtrack\n")

println("Error with Polack-Ribière = $(norm(res_polak_ribiere[:,end] - opt_f))")
println("Number of iterations = $nb_iter_polak_ribiere")
println("Number of inner loops = $nb_inner_iter_polak_ribiere")

<span style="color:#DAF7A6">
Since the initial point is on a mountain of the function landscape (i.e. near a maxima), then the first moves are very big since there is a wide range of points that are below the initial point. Thus, especially for the Polack-Ribière method, the computation of the step size using backtracking is more costly, with a total of 240 inner iterations, compared to the 52 previous ones.